In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import re  #正则表达式
import nltk 


## 读入数据 

In [2]:
data=pd.read_csv('IMDb-sample.csv',encoding='unicode_escape') #读入数据
data.info()  #查看数据信息
print(data['Sentiment'].value_counts())
data.head(10) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Index      2000 non-null   int64 
 1   URL        2000 non-null   object
 2   Text       2000 non-null   object
 3   Sentiment  2000 non-null   object
dtypes: int64(1), object(3)
memory usage: 62.6+ KB
NEG    1000
POS    1000
Name: Sentiment, dtype: int64


,Index,URL,Text,Sentiment
0,3617,http://www.imdb.com/title/tt0210075/usercomments,Girlfight follows a project dwelling New York ...,POS
1,3671,http://www.imdb.com/title/tt0337640/usercomments,Hollywood North is an euphemism from the movie...,POS
2,3157,http://www.imdb.com/title/tt0303549/usercomments,That '70s Show is definitely the funniest show...,POS
3,660,http://www.imdb.com/title/tt0716825/usercomments,"9/10- 30 minutes of pure holiday terror. Okay,...",POS
4,265,http://www.imdb.com/title/tt0182225/usercomments,"A series of random, seemingly insignificant th...",POS
5,4027,http://www.imdb.com/title/tt0347779/usercomments,A very good adaptation of the novel by amrita ...,POS
6,5820,http://www.imdb.com/title/tt0095655/usercomments,"Ah, Moonwalker, I'm a huge Michael Jackson fan...",POS
7,1574,http://www.imdb.com/title/tt0298131/usercomments,Although the beginning of the movie in New Yor...,POS
8,10668,http://www.imdb.com/title/tt0088915/usercomments,"As many reviewers here have noted, the film ve...",POS
9,1473,http://www.imdb.com/title/tt0828154/usercomments,"Bear in mind, any film (let alone documentary)...",POS


## 词形还原 

In [3]:
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None



In [ ]:
def filter_number(string_):
    str1=''
    pos=True
    all_list=[]
    for i in range(data.shape[0]):
        str1=data.iloc[i][2]
        for uchar in str1:
        # 判断是否为汉字
            if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
                continue
#          # 判断是否为数字
#         if uchar >= u'\u0030' and uchar<=u'\u0039':    
#             continue
#         # 判断是否为英文字母
# #         if (uchar >= u'\u0041' and uchar<=u'\u005a') or (uchar >= u'\u0061' and uchar<=u'\u007a'):     
# #             continue
        else:
            str1 = str1.replace(uchar, '')
            
    all_list+= ['  '.join(jieba.cut(str1))]

print(all_list)

## 预处理

In [23]:
from nltk.corpus import stopwords
wnl = nltk.stem.WordNetLemmatizer()
data['Text_words']=''
for i in range(data.shape[0]):

    text=re.sub(r'[^\w\s]','',data.iloc[i][2])  #正则表达式，去除标点符号
    text=text.lower()   
    tokenize = nltk.word_tokenize(text) #英文分词
    tokenize=[w for w in tokenize if w.isalpha() ]  #删除数字
    tokenize_text = [w for w in tokenize if not w in stopwords.words('english')] #去除停用词
    pos_tags=nltk.pos_tag(tokenize_text)#词性标注
    words_lemma=[]
    for pos in pos_tags:
        wordnet_pos = get_wordnet_pos(pos[1]) or wordnet.NOUN
        words_lemma.append(wnl.lemmatize(pos[0], pos=wordnet_pos)) # 词形还原 
    data['Text_words'].iloc[i]=words_lemma
data.head(10)

C:\Users\yaojh\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Index,URL,Text,Sentiment,Text_words
0,3617,http://www.imdb.com/title/tt0210075/usercomments,Girlfight follows a project dwelling New York ...,POS,"[girlfight, follow, project, dwell, new, york,..."
1,3671,http://www.imdb.com/title/tt0337640/usercomments,Hollywood North is an euphemism from the movie...,POS,"[hollywood, north, euphemism, movie, industry,..."
2,3157,http://www.imdb.com/title/tt0303549/usercomments,That '70s Show is definitely the funniest show...,POS,"[show, definitely, funny, show, currently, tv,..."
3,660,http://www.imdb.com/title/tt0716825/usercomments,"9/10- 30 minutes of pure holiday terror. Okay,...",POS,"[minute, pure, holiday, terror, okay, scary, s..."
4,265,http://www.imdb.com/title/tt0182225/usercomments,"A series of random, seemingly insignificant th...",POS,"[series, random, seemingly, insignificant, the..."
5,4027,http://www.imdb.com/title/tt0347779/usercomments,A very good adaptation of the novel by amrita ...,POS,"[good, adaptation, novel, amrita, pritam, urmi..."
6,5820,http://www.imdb.com/title/tt0095655/usercomments,"Ah, Moonwalker, I'm a huge Michael Jackson fan...",POS,"[ah, moonwalker, im, huge, michael, jackson, f..."
7,1574,http://www.imdb.com/title/tt0298131/usercomments,Although the beginning of the movie in New Yor...,POS,"[although, begin, movie, new, york, take, long..."
8,10668,http://www.imdb.com/title/tt0088915/usercomments,"As many reviewers here have noted, the film ve...",POS,"[many, reviewer, note, film, version, differs,..."
9,1473,http://www.imdb.com/title/tt0828154/usercomments,"Bear in mind, any film (let alone documentary)...",POS,"[bear, mind, film, let, alone, documentary, as..."


In [21]:
# 准备语料库
corpus=[]
for i in range (data.shape[0]):
    corpus.append(re.sub(r'[^\w\s]','',str(data['Text_words'][i])))
print(corpus)

['girlfight follow project dwell new york high school girl sense futility world amateur box find self esteem purpose much although film box box film much almost smell sweat technically artistically good shoot sense honesty reality girlfight chick flick rocky rather human drama even viewer dont know box able connect withgirlfight follow project dwell new york high school girl sense futility world amateur box find self esteem purpose much', 'hollywood north euphemism movie industry go canada make movie tax break cheaper cost civilized city like toronto case later vancouver peter obrian director probably saw lot invader california movie seem right way deal arrive personality try capitalize economics canada presentedneedless say moon lantern successful novel write canadian author turn flight bogota nothing original film great egotistical hasbeen michael baytes obsess happen iran offer lead part turn disasterthe film seem say many cook spoil broth seem case ultimate product save producer bo

In [22]:
#TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2))#参数默认1元
dt=vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(dt.toarray())


['abandon', 'abandon since', 'ability', 'ability see', 'able', 'able connect', 'absence', 'absence talent', 'abstract', 'abstract subject', 'account', 'account several', 'across', 'across movie', 'act', 'act general', 'act keep', 'act story', 'activity', 'activity hostel', 'actor', 'actor performer', 'actually', 'actually first', 'actually happen', 'actually magical', 'adaptation', 'adaptation novel', 'add', 'add say', 'addict', 'addict drug', 'addictive', 'addictive narcotic', 'admit', 'admit feel', 'adverse', 'adverse proportional', 'afraid', 'afraid jackson', 'agatha', 'agatha christie', 'agitate', 'agitate ring', 'ago', 'ago soon', 'ah', 'ah moonwalker', 'air', 'air one', 'album', 'album record', 'album top', 'album yeah', 'alien', 'alien resurrection', 'alienation', 'alienation bird', 'allow', 'allow desire', 'allow near', 'almost', 'almost desperate', 'almost everyone', 'almost picture', 'almost smell', 'alone', 'alone documentary', 'alone kind', 'already', 'already know', 'alrea

In [37]:
#LSA
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(12)
X1=svd.fit_transform(dt)
print(X1)

[[ 0.25316107 -0.31702836 -0.1387501  ...  0.2091579   0.
   0.        ]
 [ 0.52531618 -0.30617024 -0.0035743  ...  0.55271111  0.
   0.        ]
 [ 0.35840078  0.44954423  0.42425903 ...  0.10617192  0.
   0.        ]
 ...
 [ 0.          0.         -0.         ... -0.          0.
   0.        ]
 [ 0.          0.         -0.         ... -0.          0.
   0.        ]
 [ 0.          0.         -0.         ... -0.          0.
   0.        ]]


In [38]:
X=X1
y=data['Sentiment']

In [39]:
#分类模型
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)#留出法划分数据集
from sklearn.svm import SVC
#3个不同核函数
svc1=SVC(kernel="rbf")
svc1.fit(X_train, y_train)
score_rbf=svc1.score(X_test,y_test)
score_rbf

0.5083333333333333